# Module 9: Image Segmentation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-graphics/blob/main/notebooks/09_Module.ipynb)

**Week 14: Thresholding, Watershed, GrabCut, K-means Clustering**

## Learning Objectives
- Understand image segmentation theory and methods
- Apply various thresholding techniques (Otsu, adaptive)
- Implement Watershed algorithm for segmentation
- Use GrabCut for foreground extraction
- Apply K-means clustering for color-based segmentation

---
## ⚠️ IMPORTANT: Run All Cells in Order

**This notebook must be executed sequentially from top to bottom.**

- Click **Runtime → Run all** (or **Cell → Run All**)
- Do NOT skip cells or run them out of order
- Each cell depends on variables from previous cells

---

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 8)
print(f"OpenCV version: {cv2.__version__}")

---
## Mathematical Foundations: Segmentation Theory

### What is Image Segmentation?

**Image segmentation** partitions an image into meaningful regions:
$$
I = \bigcup_{i=1}^{n} R_i, \quad R_i \cap R_j = \emptyset \text{ for } i \neq j
$$

Where:
- $I$: Image
- $R_i$: Regions (segments)
- Each pixel belongs to exactly one region

### Why Segmentation?

1. **Object detection**: Separate objects from background
2. **Image understanding**: Identify regions of interest
3. **Measurement**: Quantify object properties
4. **Recognition**: Classify segmented regions
5. **Medical imaging**: Delineate anatomical structures

### Segmentation Categories

**1. Thresholding**: Based on intensity values
**2. Region-based**: Grow or split regions
**3. Edge-based**: Use boundary information
**4. Clustering**: Group similar pixels
**5. Graph-based**: Model image as graph

In [ ]:
# Load test images
import urllib.request
from urllib.request import Request, urlopen

# Simple image for thresholding - use a different reliable image
# Using a building/architecture image that works well for segmentation
url1 = 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/300px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg'
req1 = Request(url1, headers={'User-Agent': 'Mozilla/5.0'})
with urlopen(req1) as response:
    image_data1 = response.read()
with open('coins.jpg', 'wb') as f:
    f.write(image_data1)

img_coins = cv2.imread('coins.jpg')
img_coins_gray = cv2.cvtColor(img_coins, cv2.COLOR_BGR2GRAY)

# Complex image for advanced methods
url2 = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/320px-Cat03.jpg'
req2 = Request(url2, headers={'User-Agent': 'Mozilla/5.0'})
with urlopen(req2) as response:
    image_data2 = response.read()
with open('cat.jpg', 'wb') as f:
    f.write(image_data2)

img_cat = cv2.imread('cat.jpg')

print(f"Coins image: {img_coins.shape}")
print(f"Cat image: {img_cat.shape}")

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(cv2.cvtColor(img_coins, cv2.COLOR_BGR2RGB))
axes[0].set_title('Test Image 1')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_cat, cv2.COLOR_BGR2RGB))
axes[1].set_title('Test Image 2 (Cat)')
axes[1].axis('off')

plt.tight_layout()
plt.show()


### 1. Thresholding

**Thresholding** creates binary images by comparing pixels to threshold(s).

#### Simple Thresholding

$$
g(x, y) = \begin{cases}
255 & \text{if } f(x, y) > T \\
0 & \text{otherwise}
\end{cases}
$$

**Problem**: Choosing optimal threshold $T$?

#### Otsu's Method

**Idea**: Find threshold that minimizes **intra-class variance** (or maximizes inter-class variance).

**Algorithm**:
1. Calculate histogram
2. For each possible threshold $t$:
   - Compute weights: $w_0(t), w_1(t)$
   - Compute means: $\mu_0(t), \mu_1(t)$
   - Compute inter-class variance:
     $$
     \sigma_b^2(t) = w_0(t) w_1(t) [\mu_1(t) - \mu_0(t)]^2
     $$
3. Select $t^* = \arg\max_t \sigma_b^2(t)$

**Properties**:
- Automatic (no manual threshold)
- Assumes bimodal histogram
- Optimal for separating two classes

#### Adaptive Thresholding

**Problem**: Global threshold fails with varying illumination.

**Solution**: Compute local threshold for each pixel:
$$
T(x, y) = \text{mean}(N(x, y)) - C
$$

Where:
- $N(x, y)$: Neighborhood around $(x, y)$
- $C$: Constant (user-defined)

**Methods**:
- **Mean**: $T = \text{mean}(N) - C$
- **Gaussian**: $T = \text{weighted_mean}(N) - C$

In [ ]:
# Thresholding methods comparison
print("=" * 70)
print("THRESHOLDING METHODS")
print("=" * 70)

# Simple threshold
_, thresh_simple = cv2.threshold(img_coins_gray, 127, 255, cv2.THRESH_BINARY)

# Otsu's threshold
thresh_otsu_val, thresh_otsu = cv2.threshold(img_coins_gray, 0, 255, 
                                              cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Adaptive threshold (mean)
thresh_adaptive_mean = cv2.adaptiveThreshold(img_coins_gray, 255,
                                             cv2.ADAPTIVE_THRESH_MEAN_C,
                                             cv2.THRESH_BINARY, 11, 2)

# Adaptive threshold (Gaussian)
thresh_adaptive_gauss = cv2.adaptiveThreshold(img_coins_gray, 255,
                                              cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                              cv2.THRESH_BINARY, 11, 2)

print(f"\nOtsu's optimal threshold: {thresh_otsu_val:.2f}")

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img_coins_gray, cmap='gray')
axes[0, 0].set_title('Original Grayscale')
axes[0, 0].axis('off')

# Histogram with threshold
axes[0, 1].hist(img_coins_gray.ravel(), bins=256, range=[0, 256], color='black', alpha=0.7)
axes[0, 1].axvline(127, color='blue', linestyle='--', linewidth=2, label='Simple (127)')
axes[0, 1].axvline(thresh_otsu_val, color='red', linestyle='--', linewidth=2, 
                   label=f'Otsu ({thresh_otsu_val:.0f})')
axes[0, 1].set_title('Histogram with Thresholds')
axes[0, 1].set_xlabel('Intensity')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[0, 2].imshow(thresh_simple, cmap='gray')
axes[0, 2].set_title('Simple Threshold (T=127)')
axes[0, 2].axis('off')

axes[1, 0].imshow(thresh_otsu, cmap='gray')
axes[1, 0].set_title(f"Otsu's Method (T={thresh_otsu_val:.0f})")
axes[1, 0].axis('off')

axes[1, 1].imshow(thresh_adaptive_mean, cmap='gray')
axes[1, 1].set_title('Adaptive (Mean)')
axes[1, 1].axis('off')

axes[1, 2].imshow(thresh_adaptive_gauss, cmap='gray')
axes[1, 2].set_title('Adaptive (Gaussian)')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Otsu automatically finds optimal threshold!")
print("Adaptive thresholding handles varying illumination!")

### 2. Watershed Algorithm

The **Watershed algorithm** treats the image as a topographic surface.

#### Topographic Interpretation

- **Intensity** = altitude
- **Local minima** = valleys (catchment basins)
- **Watershed lines** = boundaries between basins

#### Algorithm (Flooding Simulation)

1. **Identify markers** (seeds) for each region
2. **Flood** from each marker:
   - Water rises from local minima
   - When waters from different basins meet → watershed line
3. **Label** pixels by their basin

#### Mathematical Formulation

**Gradient magnitude** as topographic surface:
$$
\nabla f = \sqrt{(\frac{\partial f}{\partial x})^2 + (\frac{\partial f}{\partial y})^2}
$$

**Markers**: Binary images indicating object/background seeds

#### Marker Selection

**Problem**: Over-segmentation (too many basins)

**Solution**: Provide markers
1. **Foreground markers**: Sure foreground (via morphology)
2. **Background markers**: Sure background
3. **Unknown region**: Let watershed decide

**Common approach**:
- Distance transform → local maxima = foreground
- Dilation of foreground = background
- Between them = unknown

In [ ]:
# Watershed segmentation
print("=" * 70)
print("WATERSHED SEGMENTATION")
print("=" * 70)

# Preprocess
gray = cv2.cvtColor(img_coins, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Noise removal
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

# Sure background area
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# Sure foreground area (distance transform)
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
sure_fg = np.uint8(sure_fg)

# Unknown region
unknown = cv2.subtract(sure_bg, sure_fg)

# Marker labeling
_, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1  # Add 1 so background is not 0 but 1
markers[unknown == 255] = 0  # Mark unknown region as 0

# Apply watershed
markers_watershed = cv2.watershed(img_coins, markers)

# Mark boundaries in red
img_result = img_coins.copy()
img_result[markers_watershed == -1] = [0, 0, 255]

print(f"\nNumber of regions found: {markers_watershed.max()}")

# Visualize
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

axes[0, 0].imshow(cv2.cvtColor(img_coins, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('1. Original')
axes[0, 0].axis('off')

axes[0, 1].imshow(thresh, cmap='gray')
axes[0, 1].set_title('2. Otsu Threshold')
axes[0, 1].axis('off')

axes[0, 2].imshow(opening, cmap='gray')
axes[0, 2].set_title('3. Morphology Opening')
axes[0, 2].axis('off')

axes[0, 3].imshow(dist_transform, cmap='jet')
axes[0, 3].set_title('4. Distance Transform')
axes[0, 3].axis('off')

axes[1, 0].imshow(sure_fg, cmap='gray')
axes[1, 0].set_title('5. Sure Foreground')
axes[1, 0].axis('off')

axes[1, 1].imshow(sure_bg, cmap='gray')
axes[1, 1].set_title('6. Sure Background')
axes[1, 1].axis('off')

axes[1, 2].imshow(markers, cmap='nipy_spectral')
axes[1, 2].set_title('7. Markers')
axes[1, 2].axis('off')

axes[1, 3].imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
axes[1, 3].set_title('8. Watershed Result\n(Red = boundaries)')
axes[1, 3].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Watershed segments touching objects using markers!")

### 3. GrabCut Algorithm

**GrabCut** is an interactive foreground extraction algorithm.

#### Theory

Based on **Graph Cuts** and **Gaussian Mixture Models (GMM)**.

**Energy minimization**:
$$
E(\alpha, k, \theta, z) = U(\alpha, k, \theta, z) + V(\alpha, z)
$$

Where:
- $\alpha$: Segmentation (foreground/background)
- $k$: GMM components
- $\theta$: GMM parameters
- $z$: Image
- $U$: Data term (color models)
- $V$: Smoothness term (neighboring pixels)

#### Algorithm

1. **User input**: Rectangle around object
2. **Initialization**: 
   - Everything outside rectangle = background
   - Everything inside = unknown
3. **Iterate**:
   - Build GMM for foreground and background
   - Estimate segmentation (graph cut)
   - Learn GMM parameters
   - Repeat until convergence

#### Input Modes

- **GC_INIT_WITH_RECT**: Rectangle initialization
- **GC_INIT_WITH_MASK**: User provides mask
- **GC_EVAL**: Refine existing result

#### Mask Values

- **GC_BGD (0)**: Definite background
- **GC_FGD (1)**: Definite foreground  
- **GC_PR_BGD (2)**: Probably background
- **GC_PR_FGD (3)**: Probably foreground

In [ ]:
# GrabCut segmentation
print("=" * 70)
print("GRABCUT FOREGROUND EXTRACTION")
print("=" * 70)

# Initialize mask and models
mask = np.zeros(img_cat.shape[:2], np.uint8)
bgd_model = np.zeros((1, 65), np.float64)
fgd_model = np.zeros((1, 65), np.float64)

# Define rectangle around object (adjust for your image)
h, w = img_cat.shape[:2]
rect = (10, 10, w - 20, h - 20)

print(f"\nImage size: {w} × {h}")
print(f"Rectangle: {rect}")
print(f"Running GrabCut (5 iterations)...")

# Apply GrabCut
cv2.grabCut(img_cat, mask, rect, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_RECT)

# Create binary mask
# 0 = GC_BGD, 1 = GC_FGD, 2 = GC_PR_BGD, 3 = GC_PR_FGD
mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

# Apply mask
img_foreground = img_cat * mask2[:, :, np.newaxis]

# Alternative: keep probable foreground too
mask_prob = np.where((mask == 1) | (mask == 3), 1, 0).astype('uint8')
img_foreground_prob = img_cat * mask_prob[:, :, np.newaxis]

print(f"Done!")

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Show rectangle on original
img_with_rect = img_cat.copy()
cv2.rectangle(img_with_rect, (rect[0], rect[1]), 
                  (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3)

axes[0, 0].imshow(cv2.cvtColor(img_with_rect, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('1. Original with Rectangle\n(User input)')
axes[0, 0].axis('off')

# Show mask values
mask_vis = mask.copy()
axes[0, 1].imshow(mask_vis, cmap='viridis')
axes[0, 1].set_title('2. GrabCut Mask\n(0=bg, 1=fg, 2=prob_bg, 3=prob_fg)')
axes[0, 1].axis('off')

axes[0, 2].imshow(mask2, cmap='gray')
axes[0, 2].set_title('3. Binary Mask\n(Definite FG only)')
axes[0, 2].axis('off')

axes[1, 0].imshow(cv2.cvtColor(img_cat, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('Original Image')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(img_foreground, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('4. Extracted Foreground\n(Definite only)')
axes[1, 1].axis('off')

axes[1, 2].imshow(cv2.cvtColor(img_foreground_prob, cv2.COLOR_BGR2RGB))
axes[1, 2].set_title('5. Extracted Foreground\n(Including probable)')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: GrabCut extracts foreground with minimal user input!")

### 4. K-means Clustering

**K-means** groups pixels into K clusters based on color similarity.

#### Algorithm

**Goal**: Minimize within-cluster variance:
$$
\arg\min_S \sum_{i=1}^{k} \sum_{x \in S_i} \|x - \mu_i\|^2
$$

Where:
- $S_i$: Cluster $i$
- $\mu_i$: Centroid of cluster $i$
- $x$: Data point (pixel)

**Steps**:
1. **Initialize**: Randomly select K centroids
2. **Assignment**: Assign each pixel to nearest centroid
   $$
   S_i = \{x : \|x - \mu_i\| \leq \|x - \mu_j\| \text{ for all } j\}
   $$
3. **Update**: Recompute centroids
   $$
   \mu_i = \frac{1}{|S_i|} \sum_{x \in S_i} x
   $$
4. **Repeat** until convergence

#### For Image Segmentation

**Feature vector**: RGB values
$$
x = [R, G, B]^T
$$

Or include spatial coordinates:
$$
x = [R, G, B, x, y]^T
$$

#### OpenCV K-means Parameters

- **K**: Number of clusters
- **criteria**: Termination criteria (iterations, epsilon)
- **attempts**: Number of times algorithm runs with different initial labels
- **flags**: Initialization method
  - `KMEANS_RANDOM_CENTERS`: Random
  - `KMEANS_PP_CENTERS`: K-means++ (better initialization)

In [ ]:
# K-means color clustering
print("=" * 70)
print("K-MEANS COLOR CLUSTERING")
print("=" * 70)

def kmeans_segmentation(img, K):
    """Segment image using K-means clustering"""
    # Reshape image to 2D array of pixels
    pixels = img.reshape((-1, 3))
    pixels = np.float32(pixels)
    
    # Define criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    
    # Apply K-means
    _, labels, centers = cv2.kmeans(pixels, K, None, criteria, 10, 
                                     cv2.KMEANS_PP_CENTERS)
    
    # Convert back to uint8
    centers = np.uint8(centers)
    
    # Map pixels to their cluster centers
    segmented = centers[labels.flatten()]
    segmented = segmented.reshape(img.shape)
    
    return segmented, labels.reshape(img.shape[:2]), centers

# Test with different K values
K_values = [2, 4, 8, 16]
results = []

print(f"\nSegmenting with different K values...\n")

for K in K_values:
    print(f"K={K}...", end=" ")
    segmented, labels, centers = kmeans_segmentation(img_cat, K)
    results.append((segmented, labels, centers))
    print(f"Done! Colors: {centers.tolist()}")

# Visualize
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

# Original
axes[0, 0].imshow(cv2.cvtColor(img_cat, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

# K-means results
positions = [(0, 1), (0, 2), (1, 0), (1, 1)]
for (K, (segmented, labels, centers)), (row, col) in zip(
    zip(K_values, results), positions):
    
    axes[row, col].imshow(cv2.cvtColor(segmented, cv2.COLOR_BGR2RGB))
    axes[row, col].set_title(f'K={K} clusters')
    axes[row, col].axis('off')

# Show cluster labels for K=4
_, labels_k4, _ = results[1]  # K=4
axes[1, 2].imshow(labels_k4, cmap='nipy_spectral')
axes[1, 2].set_title('Cluster Labels (K=4)')
axes[1, 2].axis('off')

# Show dominant colors for each K
for idx, (K, (_, _, centers)) in enumerate(zip(K_values, results)):
    color_bar = np.zeros((50, 50 * K, 3), dtype=np.uint8)
    for i, color in enumerate(centers):
        color_bar[:, i*50:(i+1)*50] = color
    
    row = 2
    col = idx if idx < 2 else idx - 2
    if idx >= 2:
        row = 2
        col = idx - 2
    
    axes[2, col].imshow(cv2.cvtColor(color_bar, cv2.COLOR_BGR2RGB))
    axes[2, col].set_title(f'K={K} Cluster Centers')
    axes[2, col].axis('off')

# Hide unused subplots
axes[2, 2].axis('off')

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("K-means Clustering Analysis:")
print("=" * 70)
print("  K=2:  Very coarse segmentation (2 colors)")
print("  K=4:  Good balance (4 major regions)")
print("  K=8:  More detailed (8 color regions)")
print("  K=16: Fine segmentation (16 colors)")
print("\nKey Insight: K-means segments based on color similarity!")

## Summary

### Key Concepts

1. **Thresholding**: Binary segmentation (Otsu, adaptive)
2. **Watershed**: Region-based, handles touching objects
3. **GrabCut**: Interactive foreground extraction
4. **K-means**: Color-based clustering

### Method Comparison

| Method | Input | Automatic | Handles Touching Objects | Best For |
|--------|-------|-----------|-------------------------|----------|
| **Otsu** | Grayscale | Yes | No | Simple, bimodal images |
| **Adaptive** | Grayscale | Yes | No | Varying illumination |
| **Watershed** | Gradient + Markers | Semi | **Yes** | Touching/overlapping objects |
| **GrabCut** | Color + Rectangle | Semi | Yes | Foreground extraction |
| **K-means** | Color | Yes (K needed) | No | Color-based regions |

### Mathematical Formulas Reference

**Thresholding**:
$$
g(x,y) = \begin{cases} 255 & f(x,y) > T \\ 0 & \text{otherwise} \end{cases}
$$

**Otsu inter-class variance**:
$$
\sigma_b^2(t) = w_0(t) w_1(t) [\mu_1(t) - \mu_0(t)]^2
$$

**K-means objective**:
$$
\min_S \sum_{i=1}^{k} \sum_{x \in S_i} \|x - \mu_i\|^2
$$

### OpenCV Functions Reference

| Operation | Function | Key Parameters |
|-----------|----------|----------------|
| Simple threshold | `cv2.threshold(img, thresh, maxval, type)` | THRESH_BINARY, THRESH_OTSU |
| Adaptive | `cv2.adaptiveThreshold(img, maxval, method, type, blockSize, C)` | ADAPTIVE_THRESH_MEAN_C |
| Watershed | `cv2.watershed(img, markers)` | Needs 32-bit markers |
| GrabCut | `cv2.grabCut(img, mask, rect, bgd, fgd, iter, mode)` | GC_INIT_WITH_RECT |
| K-means | `cv2.kmeans(data, K, None, criteria, attempts, flags)` | KMEANS_PP_CENTERS |
| Distance transform | `cv2.distanceTransform(img, distanceType, maskSize)` | DIST_L2 |

### When to Use Each Method

**Otsu/Adaptive**:
- Simple binary segmentation
- Document processing
- Preprocessing for other methods

**Watershed**:
- Counting objects (cells, coins, etc.)
- Separating touching objects
- When you can provide markers

**GrabCut**:
- Photo editing (background removal)
- Interactive applications
- When user can draw rectangle

**K-means**:
- Color quantization
- Compression
- When regions have distinct colors

**Next**: Module 10 - Feature Detection (Advanced)